In [ ]:
# cd into virtualhome repo
from simulation.unity_simulator import comm_unity
YOUR_FILE_NAME = "/Users/sunilbaijal/virtualhome/macos_exec.2.2.4.app"
comm = comm_unity.UnityCommunication(file_name=YOUR_FILE_NAME, port=8080, x_display="0")

# Start the first environment
comm.reset(0)
# Get an image of the first camera
success, image = comm.camera_image([0])

# Check that the image exists
print(image[0].shape)
print(success)

In [ ]:
#%matplotlib notebook
#!pip install opencv-python
#!pip install ipdb
import IPython.display
import glob
from demo.utils_demo import *
from sys import platform
import sys
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

#import virtualhome
from unity_simulator.comm_unity import UnityCommunication
from unity_simulator import utils_viz

In [ ]:
# Visualize some environments
# add modalities parameter if needed = 'normal', 'seg_class', 'seg_inst', 'depth', 'surf_normals'
comm.reset(0) # Reset to default settings
indices = [3, 32, -5, -1, -20, 15, 48, -8, 50, 17]
img_final = display_scene_cameras(comm, indices, nrows=2)
IPython.display.display(img_final)

# Chang position and rotation of the camera (h, w, theta) and add camera
#comm.add_camera(position=[-3,1.8,-4], rotation=[30, 120, 0], field_view=60)
#comm.update_camera(c-1, position=[-3,1.8,-4], rotation=[20, 120, 0], field_view=80)
# Get the last camera
#s, c = comm.camera_count()
#img_final = display_scene_cameras(comm, [c-1], nrows=1)
#IPython.display.display(img_final)

The graph is a dictionary with `nodes` and `edges`. Each node corresponds to an object and contains information such as.
- class_name: the object_name
- states: in which state the object is
- id: a number you can use to perform actions over the object 

The graph also contains bounding box and center information, which may be useful to reason about the environment layout.

## Adding Objects and Agents

In [ ]:
# Adding a cat
imgs_prev = get_scene_cameras(comm, [-4])
success, graph = comm.environment_graph();
sofa = find_nodes(graph, class_name='sofa')[-1]
print(sofa)

add_node(graph, {'class_name': 'cat', 
                   'category': 'Animals', 
                   'id': 1000, 
                   'properties': [], 
                   'states': []})
add_edge(graph, 1000, 'ON', sofa['id'])

success, message = comm.expand_scene(graph)

imgs_final = get_scene_cameras(comm, [-4])
display_grid_img(imgs_prev+imgs_final, nrows=1)
imgs_prev = imgs_final

In [ ]:
# Adding an agent
comm.reset(4)
tv_node['states'] = ['OFF']
comm.expand_scene(prev_graph)
comm.add_character('chars/Female2', initial_room='kitchen')
s, g = comm.environment_graph()
cat_id = [node['id'] for node in g['nodes'] if node['class_name'] == 'cat'][0]

script = ['<char0> [Walk] <sofa> ({})'.format(sofa['id']),
          '<char0> [Find] <cat> ({})'.format(cat_id),
          '<char0> [Grab] <cat> ({})'.format(cat_id),
          '<char0> [Sit] <sofa> ({})'.format(sofa['id'])]

success, message = comm.render_script(script=script,
                                      processing_time_limit=60,
                                      find_solution=False,
                                      image_width=320,
                                      image_height=240,  
                                      skip_animation=False,
                                      recording=True,
                                      save_pose_data=True,
                                      file_name_prefix='relax')

# Enter here the path to the video, it should be in the same location where you stored your executable 
path_video = "/Users/sunilbaijal/virtualhome/Output"
utils_viz.generate_video(input_path=path_video, prefix='relax', output_path='/Users/sunilbaijal/virtualhome/virtualhome/videos')

In [ ]:
display_vid('/Users/sunilbaijal/virtualhome/virtualhome/videos/video_normal.mp4')

Other paramters to render_script are:
- script: a list of script lines
- randomize_execution: randomly choose elements
- random_seed: random seed to use when randomizing execution, -1 means that the seed is not set
- find_solution: find solution (True) or use graph ids to determine object instances (False)
- processing_time_limit: time limit for finding a solution
- skip_execution: skip rendering, only check if a solution exists
- output_folder: folder to output renderings, default is Output/
- file_name_prefix: prefix of created files (screenshots are put to output_folder/file_name_prefix/)
- frame_rate: frame rate
- capture_screenshot: save screenshots
- image_synthesis: save depth, segmentation, flow images
- save_pose_data: save pose data
- save_scene_states: save scene states
- character_resource: path to character resource to be used
- camera_mode: automatic (AUTO), first person (FIRST_PERSON), top (PERSON_TOP), front person view (PERSON_FRONT)